### Test Dataset Create

In [ ]:
# 2021년 매월 말일 (년, 월, 일) 데이터
last_day = [20210131, 20210228, 20210331, 20210430, 20210531, 20210630, 20210731, 20210831, 20210930, 20211031, 20211130]
last = []
for day in last_day:
    date = pd.to_datetime(str(day), infer_datetime_format=True)
    last.append(date)
last_day_df = pd.DataFrame(last, columns=["OCRN_YMD"])
last_day_df["MONTH"] = last_day_df.loc[:,"OCRN_YMD"].dt.month
last_day_df["DAY"] = last_day_df.loc[:,"OCRN_YMD"].dt.day
last_day_df["WEEKDAY"] = last_day_df.loc[:,"OCRN_YMD"].dt.weekday

In [ ]:
## 격자 데이터 + 매월 말일 데이터 
last_day_input = []
Grid_ID = data_DF["GRID_ID"].unique()
for date in last_day:
    date = pd.to_datetime(str(date), infer_datetime_format=True)
    for grid in Grid_ID:
        last_day_input.append([grid, date])
grid_df = pd.DataFrame(data = last_day_input, columns=['GRID_ID', 'OCRN_YMD'])
last_day_df = pd.merge(grid_df, last_day_df, on = "OCRN_YMD", how="left")
last_day_df["OCRN_YMD"] = last_day_df["OCRN_YMD"].astype(str)
last_day_df

## SEASON_SE_NM 추가
last_day_df['SEASON_SE_NM'] = last_day_df['MONTH'].apply(season_check)

## HOLIDAY 추가 
last_day_df = pd.merge(last_day_df, hol_df, on=['OCRN_YMD'], how='left').fillna(0)

In [ ]:
# 기상정보 추가 (DAY_RAINQTY, DAY_MSNF, AVRG_WS, AVRG_HUMIDITY, AVRG_TMPRT)
last_day_df = pd.merge(last_day_df, climate_dict, on = ["OCRN_YMD"], how="left").fillna(0)

In [ ]:
# 각 사건 영향 변수 추가 (INDUSTRIAL_CNT, BAR_CNT, SENIOR_CENTER_CNT, RESTAURANT_CNT, BULID_PERMIT_CNT, ACCIDENT_AREA_CNT)

## INDUSTRIAL_CNT 추가
last_day_df = pd.merge(last_day_df, factory_df_counted, on=['GRID_ID'], how='left')

## BAR_CNT 추가
last_day_df = pd.merge(last_day_df, bar_df_counted, on=['GRID_ID'], how='left')

## SENIOR_CENTER_CNT 추가
last_day_df = pd.merge(last_day_df, senior_df_counted, on=['GRID_ID'], how='left')

## RESTAURANT_CNT 추가
last_day_df = pd.merge(last_day_df, restaurant_df_counted, on=['GRID_ID'], how='left')

## BULID_PERMIT_CNT 추가
last_day_df = pd.merge(last_day_df, build_df_counted, on=['GRID_ID'], how='left')

## ACCIDENT_AREA_CNT 추가
last_day_df = pd.merge(last_day_df, road_df_counted, on=['GRID_ID'], how='left')

In [ ]:
# 65세 미만 유동인구(ALL_POP), 65세 이상 유동인구(ELDER_POP) 컬럼 추가 

# 같은 격자, 같은 달, 같은 요일의 유동인구의 평균값 계산
pop_mean = final_DF.groupby(["GRID_ID","MONTH","WEEKDAY"], as_index=False)["ALL_POP", "ELDER_POP"].mean()

# 같은 조건의 값 추가
last_grid = last_day_df["GRID_ID"].unique()
all_pop_df = []
for m in range(1, 12):
    w = last_day_df[last_day_df["MONTH"]==m]["WEEKDAY"].unique()[0]
    for g in last_grid:
       all_pop_df.append(list(pop_mean[(pop_mean["GRID_ID"]==g) & (pop_mean["MONTH"]==m) & (pop_mean["WEEKDAY"]==w)].iloc[0, :]))
all_pop_df = pd.DataFrame(all_pop_df, columns = pop_mean.columns)
print(all_pop_df)
all_pop_add_df = all_pop_df.drop(columns=["GRID_ID", "MONTH", "WEEKDAY"], axis=1)

# 65세 미만 유동인구(ALL_POP), 65세 이상 유동인구(ELDER_POP) 추가
last_day_df = pd.concat([last_day_df, all_pop_add_df], axis=1).reset_index(drop=True)

       GRID_ID  MONTH  WEEKDAY   ALL_POP  ELDER_POP
0     378509.0    1.0      6.0  0.000000   0.000000
1     378511.0    1.0      6.0  0.105555   0.063118
2     378512.0    1.0      6.0  0.023568   0.015320
3     378513.0    1.0      6.0  0.379208   0.248920
4     378514.0    1.0      6.0  0.056562   0.011914
...        ...    ...      ...       ...        ...
5220  417516.0   11.0      1.0  0.000000   0.000000
5221  417517.0   11.0      1.0  0.126917   0.091746
5222  417518.0   11.0      1.0  0.180926   0.137904
5223  417521.0   11.0      1.0  0.000000   0.000000
5224  418518.0   11.0      1.0  0.000000   0.000000

[5225 rows x 5 columns]


In [ ]:
# Feature Skew
lam = 0.01
last_skw_features = ['AVRG_TMPRT', 'DAY_RAINQTY', 'DAY_MSNF', 'AVRG_WS', 'AVRG_HUMIDITY', 'INDUSTRIAL_CNT', 
                     'BAR_CNT', 'SENIOR_CENTER_CNT', 'RESTAURANT_CNT', 'BULID_PERMIT_CNT', 'ACCIDENT_AREA_CNT']
skew_list = skewed_check(last_day_df, last_skw_features)

# Box-Cox Transform
for col in skew_list:
  last_day_df[col] = boxcox1p(last_day_df[col], lam)

In [ ]:
# Feature Scaling
last_scaling_col = ['AVRG_TMPRT', 'DAY_RAINQTY', 'DAY_MSNF', 'AVRG_WS', 'AVRG_HUMIDITY', 'INDUSTRIAL_CNT', 'BAR_CNT', 
                    'SENIOR_CENTER_CNT', 'RESTAURANT_CNT', 'BULID_PERMIT_CNT', 'ACCIDENT_AREA_CNT']
for col in last_scaling_col:
    last_day_df[col] = Scaler.transform(last_day_df[col].values.reshape(-1,1))

In [ ]:
# col drop
last_day_df.drop(columns=["OCRN_YMD"], axis=1, inplace=True)

In [ ]:
# 테스트 데이터와 학습 데이터 순서 정렬
test_col = ['MONTH', 'DAY', 'WEEKDAY', 'HOLIDAY', 'SEASON_SE_NM', 
       'AVRG_TMPRT', 'DAY_RAINQTY', 'DAY_MSNF', 'AVRG_WS', 'AVRG_HUMIDITY',
       'INDUSTRIAL_CNT', 'BAR_CNT', 'SENIOR_CENTER_CNT', 'RESTAURANT_CNT',
       'BULID_PERMIT_CNT', 'ACCIDENT_AREA_CNT', 'ALL_POP', 'ELDER_POP','GRID_ID']
 
last_day_df = last_day_df[test_col]

In [ ]:
# last check
last_day_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5225 entries, 0 to 5224
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MONTH              5225 non-null   int64  
 1   DAY                5225 non-null   int64  
 2   WEEKDAY            5225 non-null   int64  
 3   HOLIDAY            5225 non-null   float64
 4   SEASON_SE_NM       5225 non-null   int64  
 5   AVRG_TMPRT         5225 non-null   float64
 6   DAY_RAINQTY        5225 non-null   float64
 7   DAY_MSNF           5225 non-null   float64
 8   AVRG_WS            5225 non-null   float64
 9   AVRG_HUMIDITY      5225 non-null   float64
 10  INDUSTRIAL_CNT     5225 non-null   float64
 11  BAR_CNT            5225 non-null   float64
 12  SENIOR_CENTER_CNT  5225 non-null   float64
 13  RESTAURANT_CNT     5225 non-null   float64
 14  BULID_PERMIT_CNT   5225 non-null   float64
 15  ACCIDENT_AREA_CNT  5225 non-null   float64
 16  ALL_POP            5225 

In [ ]:
# save csv
last_day_df.to_csv("last_day_df.csv")